In [1]:
import os
import cv2
import numpy as np

In [2]:
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons, weight_regularizer_l1=0, weight_regularizer_l2=0,bias_regularizer_l1=0, bias_regularizer_l2=0):

        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))

        self.weight_regularizer_l1 = weight_regularizer_l1
        self.weight_regularizer_l2 = weight_regularizer_l2
        self.bias_regularizer_l1 = bias_regularizer_l1
        self.bias_regularizer_l2 = bias_regularizer_l2

    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.biases

    def backward(self, dvalues):
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)

        if self.weight_regularizer_l1 > 0:
            dL1 = np.ones_like(self.weights)
            dL1[self.weights < 0] = -1
            self.dweights += self.weight_regularizer_l1 * dL1

        if self.weight_regularizer_l2 > 0:
            self.dweights += 2 * self.weight_regularizer_l2 * \
            self.weights

        if self.bias_regularizer_l1 > 0:
            dL1 = np.ones_like(self.biases)
            dL1[self.biases < 0] = -1
            self.dbiases += self.bias_regularizer_l1 * dL1

        if self.bias_regularizer_l2 > 0:
            self.dbiases += 2 * self.bias_regularizer_l2 * \
            self.biases

        self.dinputs = np.dot(dvalues, self.weights.T)

In [3]:
class Layer_Dropout:
    def __init__(self, rate):
        self.rate = 1 - rate

    def forward(self, inputs):
        self.inputs = inputs
        self.binary_mask = np.random.binomial(1, self.rate, size=inputs.shape) / self.rate
        self.output = inputs * self.binary_mask

    def backward(self, dvalues):
        self.dinputs = dvalues * self.binary_mask

In [4]:
class Activation_ReLU:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(0, inputs)

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0

In [5]:
class Activation_Softmax:
    def forward(self, inputs):
        self.inputs = inputs
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities

    def backward(self, dvalues):
        self.dinputs = np.empty_like(dvalues)

        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
            single_output = single_output.reshape(-1, 1)
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)

In [6]:
class Activation_Sigmoid:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = 1 / (1 + np.exp(-inputs))

    def backward(self, dvalues):
        self.dinputs = dvalues * (1 - self.output) * self.output

In [7]:
class Optimizer_SGD:
    def __init__(self, learning_rate=1., decay=0., momentum=0.):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.momentum = momentum

    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))

    def update_params(self, layer):
        if self.momentum:
            if not hasattr(layer, 'weight_momentums'):
                layer.weight_momentums = np.zeros_like(layer.weights)
                layer.bias_momentums = np.zeros_like(layer.biases)
            weight_updates = self.momentum * layer.weight_momentums - self.current_learning_rate * layer.dweights
            layer.weight_momentums = weight_updates

            bias_updates = self.momentum * layer.bias_momentums - self.current_learning_rate * layer.dbiases
            layer.bias_momentums = bias_updates

        else:
            weight_updates = -self.current_learning_rate * layer.dweights
            bias_updates = -self.current_learning_rate * layer.dbiases

        layer.weights += weight_updates
        layer.biases += bias_updates


    def post_update_params(self):
        self.iterations += 1

In [8]:
class Optimizer_Adagrad:
    def __init__(self, learning_rate=1., decay=0., epsilon=1e-7):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon

    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))

    def update_params(self, layer):
        if not hasattr(layer, 'weight_cache'):
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_cache = np.zeros_like(layer.biases)

        layer.weight_cache += layer.dweights**2
        layer.bias_cache += layer.dbiases**2

        layer.weights += -self.current_learning_rate * layer.dweights / (np.sqrt(layer.weight_cache) + self.epsilon)
        layer.biases += -self.current_learning_rate * \
        layer.dbiases / (np.sqrt(layer.bias_cache) + self.epsilon)


    def post_update_params(self):
        self.iterations += 1

In [9]:
class Optimizer_RMSprop:
    def __init__(self, learning_rate=0.001, decay=0., epsilon=1e-7, rho=0.9):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
        self.rho = rho

    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))

    def update_params(self, layer):
        if not hasattr(layer, 'weight_cache'):
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_cache = np.zeros_like(layer.biases)

        layer.weight_cache = self.rho * layer.weight_cache + (1 - self.rho) * layer.dweights**2
        layer.bias_cache = self.rho * layer.bias_cache + (1 - self.rho) * layer.dbiases**2

        layer.weights += -self.current_learning_rate * layer.dweights / (np.sqrt(layer.weight_cache) + self.epsilon)

        layer.biases += -self.current_learning_rate * layer.dbiases / (np.sqrt(layer.bias_cache) + self.epsilon)

    def post_update_params(self):
        self.iterations += 1


In [10]:
class Optimizer_Adam:
    def __init__(self, learning_rate=0.001, decay=0., epsilon=1e-7, beta_1=0.9, beta_2=0.999):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
        self.beta_1 = beta_1
        self.beta_2 = beta_2

    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))

    def update_params(self, layer):
        if not hasattr(layer, 'weight_cache'):
            layer.weight_momentums = np.zeros_like(layer.weights)
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_momentums = np.zeros_like(layer.biases)
            layer.bias_cache = np.zeros_like(layer.biases)

        layer.weight_momentums = self.beta_1 * layer.weight_momentums + (1 - self.beta_1) * layer.dweights

        layer.bias_momentums = self.beta_1 * layer.bias_momentums + (1 - self.beta_1) * layer.dbiases

        weight_momentums_corrected = layer.weight_momentums / (1 - self.beta_1 ** (self.iterations + 1))

        bias_momentums_corrected = layer.bias_momentums / (1 - self.beta_1 ** (self.iterations + 1))

        layer.weight_cache = self.beta_2 * layer.weight_cache + (1 - self.beta_2) * layer.dweights**2

        layer.bias_cache = self.beta_2 * layer.bias_cache + (1 - self.beta_2) * layer.dbiases**2

        weight_cache_corrected = layer.weight_cache / (1 - self.beta_2 ** (self.iterations + 1))

        bias_cache_corrected = layer.bias_cache / (1 - self.beta_2 ** (self.iterations + 1))

        layer.weights += -self.current_learning_rate * weight_momentums_corrected / (np.sqrt(weight_cache_corrected) +self.epsilon)

        layer.biases += -self.current_learning_rate * bias_momentums_corrected / (np.sqrt(bias_cache_corrected) + self.epsilon)

    def post_update_params(self):
        self.iterations += 1

In [11]:
class Loss:
    def regularization_loss(self, layer):
        regularization_loss = 0
        if layer.weight_regularizer_l1 > 0:
            regularization_loss += layer.weight_regularizer_l1 * np.sum(np.abs(layer.weights))

        if layer.weight_regularizer_l2 > 0:
            regularization_loss += layer.weight_regularizer_l2 * np.sum(layer.weights * layer.weights)

        if layer.bias_regularizer_l1 > 0:
            regularization_loss += layer.bias_regularizer_l1 * np.sum(np.abs(layer.biases))

        if layer.bias_regularizer_l2 > 0:
            regularization_loss += layer.bias_regularizer_l2 * np.sum(layer.biases * layer.biases)

        return regularization_loss

    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        data_loss = np.mean(sample_losses)
        return data_loss


In [16]:
class Loss_CategoricalCrossentropy(Loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        if len(y_true.shape) == 1:
            y_true = y_true.astype(int)  # Convert y_true to integers
            correct_confidences = y_pred_clipped[range(samples), y_true]
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        labels = len(dvalues[0])
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]

        self.dinputs = -y_true / dvalues
        self.dinputs = self.dinputs / samples

In [13]:
class Activation_Softmax_Loss_CategoricalCrossentropy():
    def __init__(self):
        self.activation = Activation_Softmax()
        self.loss = Loss_CategoricalCrossentropy()

    def forward(self, inputs, y_true):
        self.activation.forward(inputs)
        self.output = self.activation.output
        return self.loss.calculate(self.output, y_true)

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true, axis=1)
        self.dinputs = dvalues.copy()
        self.dinputs[range(samples), y_true] -= 1
        self.dinputs = self.dinputs / samples

In [14]:
class Loss_BinaryCrossentropy(Loss):
    def forward(self, y_pred, y_true):
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        sample_losses = -(y_true * np.log(y_pred_clipped) + (1 - y_true) * np.log(1 - y_pred_clipped))
        sample_losses = np.mean(sample_losses, axis=-1)
        return sample_losses

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        outputs = len(dvalues[0])
        clipped_dvalues = np.clip(dvalues, 1e-7, 1 - 1e-7)
        self.dinputs = -(y_true / clipped_dvalues - (1 - y_true) / (1 - clipped_dvalues)) / outputs
        self.dinputs = self.dinputs / samples

In [37]:
# Define the path to the images folder
images_folder = os.path.join(os.getcwd(), 'images')

# Define the list of fruit types
fruit_types = ["apple", "banana", "cherry", "chickoo", "grapes", "kiwi", "mango", "orange", "strawberry"]

# Define the lists to store the dataset and labels
dataset = []
labels = []

# Iterate over each fruit type
for fruit_type in fruit_types:
    # Define the path to the fruit type folder
    fruit_folder = os.path.join(images_folder, fruit_type)

    # Iterate over the image files in the fruit type folder
    for image_file in os.listdir(fruit_folder):
        # Construct the image file path
        image_path = os.path.join(fruit_folder, image_file)

        # Load the image
        image = cv2.imread(image_path)

        # Check if the image is loaded successfully
        if image is not None:
            # Preprocess the image as needed (e.g., resizing, normalization)
            image = cv2.resize(image, (64, 64))
            image = image / 255.0

            # Append the preprocessed image and its corresponding label to the lists
            dataset.append(image)
            labels.append(fruit_types.index(fruit_type))
        else:
            print(f"Failed to load image: {image_path}")

# Convert the dataset and labels to numpy arrays
dataset = np.array(dataset)
labels = np.array(labels)

# Reshape the dataset to 2D array
dataset = np.reshape(dataset, (dataset.shape[0], -1))

# Shuffle the dataset and labels
shuffle_indices = np.random.permutation(len(dataset))
dataset = dataset[shuffle_indices]
labels = labels[shuffle_indices]

# Split the data into training and testing sets
train_ratio = 0.8
split_index = int(train_ratio * len(dataset))

X_train = dataset[:split_index]
Y_train = labels[:split_index]

X_test = dataset[split_index:]
Y_test = labels[split_index:]

In [363]:
# Create the model
dense1 = Layer_Dense(X_train.shape[1], 128, weight_regularizer_l2=1e-3, bias_regularizer_l2=1e-3)
activation1 = Activation_ReLU()
dropout = Layer_Dropout(0.5)  # Adjust the dropout rate as needed
dense2 = Layer_Dense(128, 64, weight_regularizer_l2=1e-3, bias_regularizer_l2=1e-3)
activation2 = Activation_ReLU()
dense3 = Layer_Dense(64, len(fruit_types))
activation3 = Activation_Softmax()

# Define the loss function
loss_function = Loss_CategoricalCrossentropy()

# Define the optimizer
optimizer = Optimizer_Adam(learning_rate=0.001, decay=1e-7)

# Training loop
for epoch in range(151):
    # Forward pass
    dense1.forward(X_train)
    activation1.forward(dense1.output)
    dropout.forward(activation1.output)
    dense2.forward(dropout.output)
    activation2.forward(dense2.output)
    dense3.forward(activation2.output)
    activation3.forward(dense3.output)

    # Calculate loss
    loss = loss_function.calculate(activation3.output, Y_train)

    # Calculate accuracy
    predictions = np.argmax(activation3.output, axis=1)
    accuracy = np.mean(predictions == Y_train)

    if epoch % 10 == 0:
        print(f"epoch: {epoch}, acc: {accuracy:.3f}, loss: {loss:.3f}")

    # Backward pass
    loss_function.backward(activation3.output, Y_train)
    activation3.backward(loss_function.dinputs)
    dense3.backward(activation3.dinputs)
    activation2.backward(dense3.dinputs)
    dense2.backward(activation2.dinputs)
    dropout.backward(dense2.dinputs)
    activation1.backward(dropout.dinputs)
    dense1.backward(activation1.dinputs)

    # Update parameters
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.update_params(dense3)
    optimizer.post_update_params()

# Testing
dense1.forward(X_test)
activation1.forward(dense1.output)
dropout.forward(activation1.output)
dense2.forward(dropout.output)
activation2.forward(dense2.output)
dense3.forward(activation2.output)
activation3.forward(dense3.output)

# Calculate the loss and accuracy on the testing set
test_loss = loss_function.calculate(activation3.output, Y_test)
test_predictions = np.argmax(activation3.output, axis=1)
test_accuracy = np.mean(test_predictions == Y_test)

print(f"Test accuracy: {test_accuracy:.3f}, Test loss: {test_loss:.3f}")

epoch: 0, acc: 0.118, loss: 2.197
epoch: 10, acc: 0.150, loss: 2.147
epoch: 20, acc: 0.300, loss: 1.919
epoch: 30, acc: 0.429, loss: 1.561
epoch: 40, acc: 0.568, loss: 1.268
epoch: 50, acc: 0.638, loss: 1.113
epoch: 60, acc: 0.659, loss: 0.928
epoch: 70, acc: 0.756, loss: 0.748
epoch: 80, acc: 0.774, loss: 0.689
epoch: 90, acc: 0.805, loss: 0.566
epoch: 100, acc: 0.861, loss: 0.458
epoch: 110, acc: 0.906, loss: 0.372
epoch: 120, acc: 0.885, loss: 0.358
epoch: 130, acc: 0.955, loss: 0.228
epoch: 140, acc: 0.934, loss: 0.241
epoch: 150, acc: 0.944, loss: 0.243
Test accuracy: 0.472, Test loss: 2.812


In [364]:
# Create the model
dense1 = Layer_Dense(X_train.shape[1], 128, weight_regularizer_l2=1e-3, bias_regularizer_l2=1e-3)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(128, 64, weight_regularizer_l2=1e-3, bias_regularizer_l2=1e-3)
activation2 = Activation_ReLU()
dense3 = Layer_Dense(64, len(fruit_types))
activation3 = Activation_Softmax()

# Define the loss function
loss_function = Loss_CategoricalCrossentropy()

# Define the optimizer
optimizer = Optimizer_Adam(learning_rate=0.001, decay=1e-7)

# Training loop
for epoch in range(401):
    # Forward pass
    dense1.forward(X_train)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    activation2.forward(dense2.output)
    dense3.forward(activation2.output)
    activation3.forward(dense3.output)

    # Calculate loss
    loss = loss_function.calculate(activation3.output, Y_train)

    # Calculate accuracy
    predictions = np.argmax(activation3.output, axis=1)
    accuracy = np.mean(predictions == Y_train)

    if epoch % 100 == 0:
        print(f"epoch: {epoch}, acc: {accuracy:.3f}, loss: {loss:.3f}")

    # Backward pass
    loss_function.backward(activation3.output, Y_train)
    activation3.backward(loss_function.dinputs)
    dense3.backward(activation3.dinputs)
    activation2.backward(dense3.dinputs)
    dense2.backward(activation2.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    # Update parameters
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.update_params(dense3)
    optimizer.post_update_params()

# Testing
dense1.forward(X_test)
activation1.forward(dense1.output)
dense2.forward(activation1.output)
activation2.forward(dense2.output)
dense3.forward(activation2.output)
activation3.forward(dense3.output)

# Calculate the loss and accuracy on the testing set
test_loss = loss_function.calculate(activation3.output, Y_test)
test_predictions = np.argmax(activation3.output, axis=1)
test_accuracy = np.mean(test_predictions == Y_test)

print(f"Test accuracy: {test_accuracy:.3f}, Test loss: {test_loss:.3f}")

epoch: 0, acc: 0.115, loss: 2.197
epoch: 100, acc: 0.916, loss: 0.354
epoch: 200, acc: 1.000, loss: 0.037
epoch: 300, acc: 1.000, loss: 0.078
epoch: 400, acc: 1.000, loss: 0.028
Test accuracy: 0.486, Test loss: 2.823


In [356]:
# Create the model
dense1 = Layer_Dense(X_train.shape[1], 64, weight_regularizer_l2=1e-3, bias_regularizer_l2=1e-3)
activation1 = Activation_ReLU()
dropout = Layer_Dropout(0.5)
dense2 = Layer_Dense(64, 32)
activation2 = Activation_ReLU()
dense3 = Layer_Dense(32, len(fruit_types))
activation3 = Activation_Softmax()

# Define the loss function
loss_function = Loss_CategoricalCrossentropy()

# Define the optimizer
optimizer = Optimizer_Adam(learning_rate=0.001, decay=1e-7)

# Training loop
for epoch in range(301):
    # Forward pass
    dense1.forward(X_train)
    activation1.forward(dense1.output)
    dropout.forward(activation1.output)
    dense2.forward(dropout.output)
    activation2.forward(dense2.output)
    dense3.forward(activation2.output)
    activation3.forward(dense3.output)

    # Calculate loss
    loss = loss_function.calculate(activation3.output, Y_train)

    # Calculate accuracy
    predictions = np.argmax(activation3.output, axis=1)
    accuracy = np.mean(predictions == Y_train)

    if epoch % 100 == 0:
        print(f"epoch: {epoch}, acc: {accuracy:.3f}, loss: {loss:.3f}")

    # Backward pass
    loss_function.backward(activation3.output, Y_train)
    activation3.backward(loss_function.dinputs)
    dense3.backward(activation3.dinputs)
    activation2.backward(dense3.dinputs)
    dense2.backward(activation2.dinputs)
    dropout.backward(dense2.dinputs)
    activation1.backward(dropout.dinputs)
    dense1.backward(activation1.dinputs)

    # Update parameters
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.update_params(dense3)
    optimizer.post_update_params()

# Testing
dense1.forward(X_test)
activation1.forward(dense1.output)
dropout.forward(activation1.output)
dense2.forward(dropout.output)
activation2.forward(dense2.output)
dense3.forward(activation2.output)
activation3.forward(dense3.output)

# Calculate the loss and accuracy on the testing set
test_loss = loss_function.calculate(activation3.output, Y_test)
test_predictions = np.argmax(activation3.output, axis=1)
test_accuracy = np.mean(test_predictions == Y_test)

print(f"Test accuracy: {test_accuracy:.3f}, Test loss: {test_loss:.3f}")

epoch: 0, acc: 0.105, loss: 2.197
epoch: 100, acc: 0.746, loss: 0.743
epoch: 200, acc: 0.923, loss: 0.292
epoch: 300, acc: 0.958, loss: 0.193
Test accuracy: 0.486, Test loss: 2.753


In [353]:
# Create the model
dense1 = Layer_Dense(X_train.shape[1], 32, weight_regularizer_l2=5e-4, bias_regularizer_l2=5e-4)
activation1 = Activation_ReLU()
dropout = Layer_Dropout(0.5)  # Adjust the dropout rate as needed
dense2 = Layer_Dense(32, len(fruit_types))
activation2 = Activation_Softmax()

# Define the loss function
loss_function = Loss_CategoricalCrossentropy()

# Define the optimizer
optimizer = Optimizer_Adam(learning_rate=0.001, decay=1e-7)

# Training loop
for epoch in range(901):
    # Forward pass
    dense1.forward(X_train)
    activation1.forward(dense1.output)
    dropout.forward(activation1.output)
    dense2.forward(dropout.output)
    activation2.forward(dense2.output)

    # Calculate loss
    loss = loss_function.calculate(activation2.output, Y_train)

    # Calculate accuracy
    predictions = np.argmax(activation2.output, axis=1)
    accuracy = np.mean(predictions == Y_train)

    if epoch % 100 == 0:
        print(f"epoch: {epoch}, acc: {accuracy:.3f}, loss: {loss:.3f}")

    # Backward pass
    loss_function.backward(activation2.output, Y_train)
    activation2.backward(loss_function.dinputs)
    dense2.backward(activation2.dinputs)
    dropout.backward(dense2.dinputs)
    activation1.backward(dropout.dinputs)
    dense1.backward(activation1.dinputs)

    # Update parameters
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()

# Testing
dense1.forward(X_test)
activation1.forward(dense1.output)
dense2.forward(activation1.output)
activation2.forward(dense2.output)

# Calculate the loss and accuracy on the testing set
test_loss = loss_function.calculate(activation2.output, Y_test)
test_predictions = np.argmax(activation2.output, axis=1)
test_accuracy = np.mean(test_predictions == Y_test)

print(f"Test accuracy: {test_accuracy:.3f}, Test loss: {test_loss:.3f}")

epoch: 0, acc: 0.122, loss: 2.196
epoch: 100, acc: 0.498, loss: 1.253
epoch: 200, acc: 0.645, loss: 0.889
epoch: 300, acc: 0.742, loss: 0.649
epoch: 400, acc: 0.711, loss: 0.610
epoch: 500, acc: 0.746, loss: 0.559
epoch: 600, acc: 0.798, loss: 0.477
epoch: 700, acc: 0.808, loss: 0.429
epoch: 800, acc: 0.756, loss: 0.545
epoch: 900, acc: 0.767, loss: 0.515
Test accuracy: 0.514, Test loss: 2.492


In [350]:
import cv2
import numpy as np

# Load and preprocess the image
image_path = r"C:\Users\armin\PycharmProjects\artificial_intelligence\images\mango\Image_40.jpg"  # Replace with the path to your image
image = cv2.imread(image_path)
image = cv2.resize(image, (64, 64))
image = image / 255.0
image = np.reshape(image, (1, -1))

# Forward pass through the model
dense1.forward(image)
activation1.forward(dense1.output)
dense2.forward(activation1.output)
activation2.forward(dense2.output)

# Get the predicted class index
prediction = np.argmax(activation2.output, axis=1)

# Map the predicted class index to the fruit name
predicted_fruit = fruit_types[prediction[0]]

# Print the predicted fruit
print(f"Predicted fruit: {predicted_fruit}")

Predicted fruit: mango
